In [1]:
# !sudo pip install h5py

In [1]:
import copy
import datetime
import numpy as np
import time
import pandas as pd
import pandas_datareader.data as web
from sklearn.cluster import KMeans
from sklearn import preprocessing
import plotly
import itertools
from talib.abstract import *
import os
import sys
from keras.models import model_from_json
from keras.models import model_from_yaml
import requests
import numpy as np
import pandas as pd
import h5py
from keras.models import Sequential
from keras.layers import Dense
from sklearn.externals import joblib
print("Import Function for the Neural Network Done")
from multiprocessing import Pool


# import plotly.plotly as py
# from plotly.graph_objs import *
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# import plotly.graph_objs as go
# plotly.tools.set_credentials_file(username='maiti.t', api_key='km8Kdfszic1Cu6ZAWiZJ')
# plotly.offline.init_notebook_mode(connected=True)

Using TensorFlow backend.


Import Function for the Neural Network Done


In [2]:
import pickle
fileDir = os.path.dirname(os.path.realpath('__file__'))
print(' Local Path defined')
print(fileDir)

 Local Path defined
/home/mohan


In [3]:
def get_open_normalised_prices(df, start, end):
    
    print('************************************************************************')
    df1 = pd.DataFrame(df["High"]/df["Open"])
    df1.columns=["H/O"]
    df1["L/O"] = df["Low"]/df["Open"]
    df1["C/O"] = df["Close"]/df["Open"]
    
    return df1

# create_follow_cluster_matrix

In [4]:
def create_follow_cluster_matrix(data,k):
    
    print('************************************************************************')
    data["ClusterTomorrow"] = data["Cluster"].shift(-1)
    data.dropna(inplace=True)
    data["ClusterTomorrow"] = data["ClusterTomorrow"].apply(int)
#     print("Tesssssssssssttttttttttttttt")
#     print(zip(data["Cluster"], data["ClusterTomorrow"]))
    data["ClusterMatrix"] = list(zip(data["Cluster"], data["ClusterTomorrow"]))
    cmvc = data["ClusterMatrix"].value_counts()
    clust_mat = np.zeros( (k, k) )
    for row in cmvc.iteritems():
#         print(row[1]*100.0/len(data))
        clust_mat[row[0]] = row[1]*100.0/len(data)
    print("Cluster Follow-on Matrix:")
    print(clust_mat)
    return clust_mat,cmvc

In [5]:
def get_indicators(stocks, period):
#     for i in stocks:
        print("***********************************************************************")
        stocks.columns = [s.lower() for s in stocks.columns]
        print("Adding Features: Started  ")
        features = pd.DataFrame(SMA(stocks, timeperiod=10))
        features.columns = ['sma_10']
        features['NATR']=pd.DataFrame(NATR(stocks, timeperiod=14))
        features = pd.concat([features,STOCHF(stocks, fastk_period=14, fastd_period=3)], axis=1)
        features['willr'] = pd.DataFrame(WILLR(stocks, timeperiod=14))
        features['rsi'] = pd.DataFrame(RSI(stocks, timeperiod=14))
        features['wma_10'] = pd.DataFrame(WMA(stocks,10))
        features['T3'] =pd.DataFrame(T3(stocks, timeperiod=5, vfactor=0))
        features['closePrice']=pd.DataFrame(stocks['close'].shift(-period))
        features['return_pct_change'] = ROC(stocks, timeperiod=period)
        features['return_pct_change'] = features['return_pct_change'].shift(-period)
        features['pct_change'] = features['return_pct_change'].apply(lambda x: '1' if x > 0 else '0' if x <= 0 else np.nan)
        features = features.dropna()
        return features

In [6]:
def modellingNeuralNetwork(trainX,trainY,numbFeatures,i):
    print("Creating Neural Network .....")
    starttime= time.time()
    model = Sequential()
    model.add(Dense(5, input_dim=numbFeatures, activation='relu'))
    model.add(Dense(11, input_dim=numbFeatures, activation='relu'))
    model.add(Dense(11, input_dim=numbFeatures, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'mape'])
    history1 = model.fit(trainX, trainY, epochs=200, batch_size=2, verbose=0)
    print("Time Taken to Model %s seconds ---" % (time.time() - starttime))
    print("*******************************************")
    print(history1)
    print("*******************************************")
    scores = model.evaluate(trainX,trainY,verbose = 0)
#     scores = model.evaluate(X, Y, verbose=0)
#     print("%s: %.2f%%" % (model.metrics_names[0], scores[1]*100))
    print(model.summary())
    # serialize model to JSON
    model_json = model.to_json()
    with open(str(stock)+"model"+str(i)+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    h5filename = str(stock)+'model'+str(i)+'.h5'
    model.save_weights(h5filename)
    print("Saved model to disk")
 

    return model,history1

In [7]:
def create_dataset1(datasetReal,i):
    print(type(datasetReal))
    xdataset = datasetReal.copy()
    xdataset = xdataset[xdataset.columns[xdataset.columns !='closePrice']]
    dataset = np.array(xdataset)
    print("Total Number of features in trainig Dataset "+ str(xdataset.columns.size))
    dataX = [dataset[n] for n in range(len(dataset)-i)]
    return np.array(dataX),xdataset.columns.size

In [8]:
predictedClosePrice1 = []
frameshistory1=[]
# model = []
# def testingModel1(inputClusterone):
    
        
              
        
#         for i in range(1,3):
#             print("Training the Neural Network for next "+ str(i)+" Prediction")
#             starttime= time.time() 
            
#             print("Copying the input Dataset ")
#             inputdata = inputClusterone.copy()
            
#             dataset1 = get_indicators(inputClusterone,i)

#             print("Making The Training Dataset")
#             traindataset = dataset1.head(len(dataset1)-2)
#             print("CLosePrice : Current Trade Date: " + str(traindataset['closePrice']))
#             trainY = np.array(traindataset['closePrice'])[i:]
#             trainX, numbFeatures = create_dataset1(traindataset,i)
            
#             m,h = modellingNeuralNetwork(trainX,trainY,numbFeatures,i)
            
# #             f={"models":m}
# #             s = pickle.dumps( m, "model"+str(i)+".pkl" ) 
# #             s = pickle.dumps(m) 
# #             joblib.dump(s, 'model'+str(i)+'.pkl')
#             print("Model Saved")
# #             joblib.dump(m, '"model"+str(i)+".pkl"') 
# #             model.append(m)
            
#             fullReport  = pd.DataFrame(h.history)
#             fullReport.to_csv("Day_"+ str(i)+" NN_PredictionModel.csv")
#             finalMAPE=fullReport['mean_absolute_percentage_error'].tail(1)
#             finalLoss = fullReport['loss'].tail(1)
#             finalMAE = fullReport['mean_absolute_error'].tail(1)

#             finalreport = pd.DataFrame(finalMAPE)
#             finalreport.columns = ['MAPE']
#             finalreport['Loss']=finalLoss
#             finalreport['MAE']=finalMAE
#             finalreport.index = range(1,len(finalreport)+1)

#             frameshistory.append(finalreport)
#             print(finalreport)

#             print("Testing on the Latest Trade Data ...")

# #             trainfeature=traindataset.tail(len(traindataset)-1)
# #             trainlabel= trainfeature['closePrice']

# #             prediction = m.predict(trainX)
# #             my_list = map(lambda x: x[0], prediction)
# #             traintestlabel = pd.Series(my_list)

#             testdata = dataset1.tail(1)
# #             testdata = testDataLatestTrade
# #             testexpectedlabel = testdata['closePrice']

#             xtest = testdata[testdata.columns[testdata.columns !='closePrice']]
#             ## Predicting the Future Price 
#             print("-----------------------------------------")
#             print("Test record Given")
#             print(xtest)
#             print("----------------------------------")
#             predictiontest = m.predict(np.array(xtest))
            
            
#             temp_list = map(lambda x: x[0], predictiontest)
#             testlabel = pd.Series(temp_list)
#             print("**********************************************************")
# #             print("Excpected Close Price "+ str( testexpectedlabel.values))
#             print("Predicted Close Price  "+ str(testlabel[0]))
# #             predictedClosePrice.append(testlabel[0])
#             clf = joblib.load('predictedClosePrice1.pkl') 
#             clf.append(testlabel[0])
#             joblib.dump(clf, 'predictedClosePrice1.pkl')
#             print("**********************************************************")
#             print("Time Taken to Model %s seconds ---" % (time.time() - starttime))
            
#         print("Testing the Model Finished ...")
#         allrecord = pd.concat(frameshistory)
#         allrecord.index = range(1,len(allrecord)+1)
#         print("Final Model Evaluation Scores ")
#         print(allrecord)


In [9]:
def datadownload(start,end,stock):
    try:
        dow = web.DataReader(stock, "yahoo", start, end)
        dow.to_csv(fileDir+ '/stockMarketModels/'+str(stock)+'/'+"All"+stock+"data.csv")
    except (RuntimeError, TypeError, NameError,Exception):
                   print("Oops! Yahoo Data Service is not working ....")
                   print("Data Taken from last updated repository")
                   dow=pd.read_csv(fileDir+ '/stockMarketModels/'+str(stock)+'/'+"All"+stock+"data.csv")
        
    
    return dow
    

In [10]:
def knn(dow_norm,k):
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(dow_norm)
    labels = km.labels_
    return labels,km

In [11]:
def getinformationCluster(cmvc,km):
    print("Ranking of Clusters ");
    print(str(cmvc))
    print('************************************************************************')
    print("Centroid for the K Means Clusters ")
    kmean = pd.DataFrame(km.cluster_centers_)
    kmean.columns=['X','Y','Z']
    print(kmean)
    print('************************************************************************')
    

In [12]:
def predictCluster(km,dow_norm,cmvc,clust_mat):
    print("Getting the Latest Trade OHLC Values for DOW 30 ") 
    testdoeN = dow_norm.copy()
    print("*****************************************8")
#     print(testdoeN.tail(5))
#     del testdoeN['Cluster']
#     print("Testtttttt")
    
    print(testdoeN.tail(1).values.reshape(-1,3))
    predictedCluster = km.predict(testdoeN.tail(1).values.reshape(-1,3))[0]
    print("Predicted Cluster of Last Trade OHLC Date "+ " "+ str(testdoeN.index[len(testdoeN)-1])+" "+ str(predictedCluster))
    predictedNextCluster = []
    for i,row in cmvc.iteritems():
        if(i[0]==predictedCluster):
            predictedNextCluster.append(i[1])
    print("Predicted Next Day Cluster would be : "+ str(predictedNextCluster))
    
    probabCluster=[]
    totalPrbablityfromPast=0
    for i in predictedNextCluster:
            totalPrbablityfromPast=totalPrbablityfromPast+clust_mat[predictedCluster][i]
    #         probabCluster.append(clust_mat[predictedCluster][i])
    print("Calulating the Relative Probablity ")
    for i in predictedNextCluster:
            probabCluster.append(clust_mat[predictedCluster][i]/totalPrbablityfromPast*100)
            
    nextClusterMatrix = pd.DataFrame(predictedNextCluster)
    nextClusterMatrix.columns=["Cluster"]
    nextClusterMatrix['Probability']=probabCluster
    
    print("Next Day CLuster and its Relative probablity from the Predicted Cluster : "+ str(predictedCluster))
    print(nextClusterMatrix)
    return nextClusterMatrix
    


In [13]:
def getRightClusteredData(nextClusterMatrix,dowtrain):
    print("Getting the filtered Past data where the maximum predicted range of the next DAY OHLC would be.. ")
    inputClusterone = dowtrain[dowtrain['Cluster']==nextClusterMatrix['Cluster'][0]]
    return inputClusterone
    

In [14]:
# import pandas_datareader.data as web
# start = datetime.datetime(2005, 1, 1)
# end =  time.strftime("%Y-%m-%d")
# dow = web.DataReader("^DJI", "yahoo", start, end)


In [15]:
# dow=pd.read_csv("AllDJIdata.csv")

In [16]:
# frameshistory=[]
def testingModel(i):
#         for i in range(1,3):
            print("Training the Neural Network for next "+ str(i)+" Days Prediction")
            starttime= time.time() 
            print("Copying the input Dataset for "+str(i)+" Days Prediction")
            inputClusterone = joblib.load(fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'inputClusterone.pkl')
            
            inputdata = inputClusterone.copy()
            
            dataset1 = get_indicators(inputClusterone,i)
            if(i ==1):
                print("Getting the features for Test data ")
                testdata = dataset1.tail(1)
                joblib.dump(testdata, fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'testdata1.pkl')
            
            print("**********Checking Process")
            print(inputdata.tail(5))
            print("_________________________________________")
            print(dataset1.tail(5))

            print("Making The Training Dataset")
            traindataset = dataset1.head(len(dataset1)-2)

            trainY = np.array(traindataset['closePrice'])[i:]
            trainX, numbFeatures = create_dataset1(traindataset,i)
            
            #################Modelling Neural Network ##########################
            m,h = modellingNeuralNetwork(trainX,trainY,numbFeatures,i)
            
 
            
            print("All Models Saved ....")
            
            fullReport  = pd.DataFrame(h.history)
            fullReport.to_csv(fileDir+ '/stockMarketModels/'+str(stock)+'/'+"Day_"+ str(i)+str(stock)+" NN_PredictionModel.csv")
            finalMAPE=fullReport['mean_absolute_percentage_error'].tail(1)
            finalLoss = fullReport['loss'].tail(1)
            finalMAE = fullReport['mean_absolute_error'].tail(1)

            finalreport = pd.DataFrame(finalMAPE)
            finalreport.columns = ['MAPE']
            finalreport['Loss']=finalLoss
            finalreport['MAE']=finalMAE
            finalreport.index = range(1,len(finalreport)+1)
            fh = joblib.load(fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'frameshistory1.pkl') 
            fh.append(finalreport)
            joblib.dump(fh, fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'frameshistory1.pkl')
            
            print(finalreport)

            print("Testing on the Latest Trade Data ...")

#             trainfeature=traindataset.tail(len(traindataset)-1)
#             trainlabel= trainfeature['closePrice']

#             prediction = m.predict(trainX)
#             my_list = map(lambda x: x[0], prediction)
#             traintestlabel = pd.Series(my_list)

#             testdata = dataset1.tail(1)
            testdata = joblib.load(fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'testdata1.pkl')
#             testdata = testDataLatestTrade
#             testexpectedlabel = testdata['closePrice']

            xtest = testdata[testdata.columns[testdata.columns !='closePrice']]
            print("-----------------------------------------")
            print("Test record Given")
            print(xtest)
            print("----------------------------------")
            predictiontest = m.predict(np.array(xtest))
            print("*****************RESULT******************************************")
            print("Current trade closing price "+ str(testdata['closePrice']))
            
            temp_list = map(lambda x: x[0], predictiontest)
            testlabel = pd.Series(temp_list)
            
#             print("Excpected Close Price "+ str( testexpectedlabel.values))
            print("Predicted Close Price  "+ str(testlabel[0]))
            print("**********************************************************")
    
            clf = joblib.load(fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'predictedClosePrice1.pkl') 
            data = pd.DataFrame()
            x= [i]
            datadf = pd.DataFrame(x)
            datadf.columns =['Day']
            datadf['PredictedClosePrice']=testlabel[0]
            clf.append(datadf)
            joblib.dump(clf, fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'predictedClosePrice1.pkl')
#             predictedClosePrice.append(testlabel[0])
            print("**********************************************************")
            print("Time Taken to Model %s seconds ---" % (time.time() - starttime))
            


In [17]:
def runprogram(stock):
    # Obtain DOW 30  pricing data from Yahoo Finance
    print('************************************************************************')
    print("Step 1:Downloading the "+str(stock)+"  data from Yahoo Finance ")
    starttime= time.time()
    start = datetime.datetime(2005, 1, 1)
    end =  time.strftime("%Y-%m-%d")
#     dow = web.DataReader("^DJI", "yahoo", start, end)
    
    dow = datadownload(start,end,stock)
        

            
    print("Status ::Data Download Finish....")
    print('************************************************************************')
    print("Step 2:Normalising the Data with respect to Daily Open Stock Price ")
    
    dow_norm = get_open_normalised_prices(dow, start, end)
    print("Number of Chosen Cluster : 5 ")
    
    k = 5
    print('************************************************************************')
    print("Step 3: KNN Process Started ...")
    
    # Deleting the last row from the KNN Dataset 
    traindowNorm = dow_norm.copy()
    traindowNorm.drop(traindowNorm.index[len(traindowNorm)-1])
    
    # Reatining the latest Trade info as normalised one
    testDataLatestTrade = dow_norm.tail(1)
    
    labels,km = knn(traindowNorm,k)
    
    dowtrain = dow.copy()
    dowtrain.drop(dowtrain.index[len(dowtrain)-1])
    dowtrain["Cluster"] = labels
    traindowNorm["Cluster"] = labels
    
    print("Status : KNN Process Finished ...")
    # Create and output the cluster follow-on matrix
    clust_mat,cmvc = create_follow_cluster_matrix(dowtrain,k)
    
    print('************************************************************************')
    
    getinformationCluster(cmvc,km)
    
    nextClusterMatrix = predictCluster(km,testDataLatestTrade,cmvc,clust_mat)
    
    inputClusterone = getRightClusteredData(nextClusterMatrix,dowtrain)
    
    inputClusterone.append(dow.tail(1))
    print("Input Data ::::::::::::::::::::::::::::::::::::::::::")
    print(inputClusterone.tail(1))
    print(":::::::::::::::::::::::::::::::::::::::::::::::::::::")
    joblib.dump(predictedClosePrice1,fileDir+'/stockMarketModels/'+str(stock)+'/'+str(stock)+'predictedClosePrice1.pkl')
    joblib.dump(frameshistory1, fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'frameshistory1.pkl')
    joblib.dump(inputClusterone, fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'inputClusterone.pkl')
    
    
    p = Pool(20)
#     frameshistory=[]
    p.map(testingModel, range (1,8))
#     testingModel1(inputClusterone)
    print("Testing the Model Finished ...")
    fh1 = joblib.load(fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'frameshistory1.pkl')
    allrecord = pd.concat(fh1)
    allrecord.index = range(1,len(allrecord)+1)
    print("Final Model Evaluation Scores ")
    print(allrecord)
    print("Total Time Taken to Model %s seconds ---" % (time.time() - starttime))
    
 


In [18]:
def makefiles(stocknames):
    for i in stocknames:
        if(not os.path.exists(fileDir+'/stockMarketModels/'+str(i))):
                os.makedirs(fileDir+'/stockMarketModels/'+str(i)) 

stocknames=["AAPL","T","BAC","JNJ","XOM"]
        
makefiles(stocknames)

for i in stocknames:
    stock = i  ## TECHNOLOGY   ## APPLE 
    runprogram(i)
    
# stock = "T"   ## TELECOMMUNICATION ## AT&T Inc.
# runprogram("T")
# stock = "BAC"    ## FINANCE BANK OF AMERICA
# runprogram("BAC")
# stock = "JNJ" ## HEALTH CARE - ## JOHNSON AND JHONSON
# runprogram("JNJ")
# stock = "XOM"  ## ENERGY - ## Exxon Mobil Corporation
# runprogram("XOM")

************************************************************************
Step 1:Downloading the AAPL  data from Yahoo Finance 
Status ::Data Download Finish....
************************************************************************
Step 2:Normalising the Data with respect to Daily Open Stock Price 
************************************************************************
Number of Chosen Cluster : 5 
************************************************************************
Step 3: KNN Process Started ...
Status : KNN Process Finished ...
************************************************************************
Cluster Follow-on Matrix:
[[  4.84429066   1.54136521   6.57439446   6.00817867   1.10097515]
 [  1.44699591   0.53475936   1.16388802   0.94369299   0.47184649]
 [  5.19031142   1.06951872   7.04624096  13.40044039   0.78641082]
 [  7.95847751   0.88078012  11.60742372  23.52941176   0.56621579]
 [  0.59767222   0.53475936   1.10097515   0.69204152   0.40893363]]
****************

2017-08-16  27671600        3                0        (3, 0)  
Getting the features for Test data 
***********************************************************************
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2017-08-03  157.050003  157.210007  155.020004  155.570007  154.961472   
2017-08-04  156.070007  157.399994  155.690002  156.389999  155.778259   
2017-08-14  159.320007  160.210007  158.750000  159.850006  159.850006   
2017-08-15  160.660004  162.199997  160.139999  161.600006  161.600006   
2017-08-16  161.940002  162.509995  160.149994  160.949997  160.949997   

              Volume  Cluster  ClusterTomorrow ClusterMatrix  
Date                                                          
2017-08-03  27097300        3                3        (3, 3)  
2017-08-04  20559900        3                2        (3, 2)  
2017-08-14  22028200        3                3        (3, 

2017-08-15  155.417640  152.134429  160.949997          -0.402233          0  Creating Neural Network .....
Total Number of features in trainig Dataset 10

Creating Neural Network .....
Total Number of features in trainig Dataset 10
Making The Training Dataset
<class 'pandas.core.frame.DataFrame'>
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2017-08-03  157.050003  157.210007  155.020004  155.570007  154.961472   
2017-08-04  156.070007  157.399994  155.690002  156.389999  155.778259   
2017-08-14  159.320007  160.210007  158.750000  159.850006  159.850006   
2017-08-15  160.660004  162.199997  160.139999  161.600006  161.600006   
2017-08-16  161.940002  162.509995  160.149994  160.949997  160.949997   

              Volume  Cluster  ClusterTomorrow ClusterMatrix  
Date                                                          
2017-08-03  27097300        3                3        (3

*******************************************
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_2 (Dense)              (None, 11)                66        
_________________________________________________________________
dense_3 (Dense)              (None, 11)                132       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 12        
Total params: 265
Trainable params: 265
Non-trainable params: 0
_________________________________________________________________
None
Saved model to disk
All Models Saved ....
       MAPE       Loss       MAE
1  4.524838  10.244695  2.282641
Testing on the Latest Trade Data ...
-----------------------------------------
Test record Given
                sma_10

*******************************************
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_2 (Dense)              (None, 11)                66        
_________________________________________________________________
dense_3 (Dense)              (None, 11)                132       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 12        
Total params: 265
Trainable params: 265
Non-trainable params: 0
_________________________________________________________________
None
Saved model to disk
All Models Saved ....
       MAPE      Loss       MAE
1  3.050526  4.588755  1.508158
Testing on the Latest Trade Data ...
-----------------------------------------
Test record Given
                sma_10  

...

2017-08-15  132.945455  132.752351           0.592287          1  
----------------------------------
*****************RESULT******************************************
Current trade closing price Date
2017-08-15    134.169998
Name: closePrice, dtype: float64
Predicted Close Price  136.719970703
**********************************************************
**********************************************************
Time Taken to Model 267.608597994 seconds ---
Time Taken to Model 310.341634035 seconds ---
*******************************************
*******************************************
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_2 (Dense)              (None, 11)                66        
_________________________________________________________________
dense_3 (D

Training the Neural Network for next 1 Days Prediction
Training the Neural Network for next 2 Days Prediction
Training the Neural Network for next 5 Days Prediction
Training the Neural Network for next 4 Days Prediction
Training the Neural Network for next 3 Days Prediction
Training the Neural Network for next 6 Days Prediction
Training the Neural Network for next 7 Days Prediction
Copying the input Dataset for 5 Days Prediction
Copying the input Dataset for 2 Days Prediction
Copying the input Dataset for 4 Days Prediction
Copying the input Dataset for 7 Days Prediction
Copying the input Dataset for 1 Days Prediction
Copying the input Dataset for 3 Days Prediction
Copying the input Dataset for 6 Days Prediction
***********************************************************************
***********************************************************************
Adding Features: Started  
***********************************************************************
Adding Features: Started  
*********

2017-08-03  80.558727  80.738785   79.959999          -0.658466          0                   Open       High        Low      Close  Adj Close    Volume  \
Date                                                                          
2017-08-01  80.160004  80.540001  79.970001  80.169998  79.400383  10421200   
2017-08-02  79.900002  80.639999  79.800003  80.599998  79.826256  10410100   
2017-08-03  80.339996  80.820000  80.300003  80.489998  79.717308  10136700   
2017-08-07  80.169998  80.169998  79.860001  80.160004  79.390480   7893800   
2017-08-08  80.019997  80.220001  79.830002  79.959999  79.192398   9028400   

            Cluster  ClusterTomorrow ClusterMatrix  
Date                                                
2017-08-01        4                4        (4, 4)  
2017-08-02        4                4        (4, 4)  
2017-08-03        4                0        (4, 0)  
2017-08-07        4                4        (4, 4)  
2017-08-08        4                0        (4, 0)  

Creating Neural Network .....
Total Number of features in trainig Dataset 10
Creating Neural Network .....
Time Taken to Model 231.995400906 seconds ---
*******************************************
*******************************************
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_2 (Dense)              (None, 11)                66        
_________________________________________________________________
dense_3 (Dense)              (None, 11)                132       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 12        
Total params: 265
Trainable params: 265
Non-trainable params: 0
_________________________________________________________________
None
Saved model to disk
All Models

**********************************************************
**********************************************************
Time Taken to Model 236.450130939 seconds ---
Time Taken to Model 296.365674973 seconds ---
*******************************************
*******************************************
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_2 (Dense)              (None, 11)                66        
_________________________________________________________________
dense_3 (Dense)              (None, 11)                132       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 12        
Total params: 265
Trainable params: 265
Non-trainable params: 0
____________________________________________

In [135]:
clf = joblib.load(fileDir+ '/stockMarketModels/'+str(stock)+'/'+str(stock)+'predictedClosePrice1.pkl')

In [136]:
clf

[   Day  PredictedClosePrice
 0    3             79.25415,    Day  PredictedClosePrice
 0    1            78.959068,    Day  PredictedClosePrice
 0    2            78.048294]